# Model checking

In [1]:
import stormvogel.model
from stormvogel import show, model_checking, property_builder

In [2]:
mdp = stormvogel.model.new_mdp("Monty Hall")

init = mdp.get_initial_state()

# first choose car position
init.set_choice(
    [(1 / 3, mdp.new_state("carchosen", {"car_pos": i})) for i in range(3)]
)

# we choose a door in each case
for s in mdp.get_states_with_label("carchosen"):
    s.set_choice(
        [
            (
                mdp.action(f"open{i}"),
                mdp.new_state("open", s.features | {"chosen_pos": i}),
            )
            for i in range(3)
        ]
    )

# the other goat is revealed
for s in mdp.get_states_with_label("open"):
    car_pos = s.features["car_pos"]
    chosen_pos = s.features["chosen_pos"]
    other_pos = {0, 1, 2} - {car_pos, chosen_pos}
    s.set_choice(
        [
            (
                1 / len(other_pos),
                mdp.new_state("goatrevealed", s.features | {"reveal_pos": i}),
            )
            for i in other_pos
        ]
    )

# we must choose whether we want to switch
for s in mdp.get_states_with_label("goatrevealed"):
    car_pos = s.features["car_pos"]
    chosen_pos = s.features["chosen_pos"]
    reveal_pos = s.features["reveal_pos"]
    other_pos = list({0, 1, 2} - {reveal_pos, chosen_pos})[0]
    s.set_choice(
        [
            (
                mdp.action("stay"),
                mdp.new_state(
                    ["done"] + (["target"] if chosen_pos == car_pos else []),
                    s.features | {"chosen_pos": chosen_pos},
                ),
            ),
            (
                mdp.action("switch"),
                mdp.new_state(
                    ["done"] + (["target"] if other_pos == car_pos else []),
                    s.features | {"chosen_pos": other_pos},
                ),
            ),
        ]
    )

# we add self loops to all states with no outgoing transitions
mdp.add_self_loops()

show.show(mdp, show_editor=True)

Output()

<IPython.core.display.Javascript object>

Output()

Output()

We can do model checking on this model by only using stormvogel functions directly. Behind the scenes this calls the stormpy model checker. Before we use the model checker however, we need to specify the task. This is done using a property string. We have provided an easy way to create them for beginner users.

In [3]:
property_builder.build_property_string(mdp)

Output()

Output()

Create a property string and give it as argument to the model checking function in the following way

In [4]:
result = model_checking.model_checking(mdp,'Pmax=? [F "done"]', True) #true lets it return a scheduler as well

In [5]:
result.scheduler.taken_actions[1] = mdp.action(f"open1")

In [6]:
result.scheduler.model

Model(name=None, type=<ModelType.MDP: 2>, states={0: State(labels=['init'], features={}, id=0, model=..., observation=None, name='0'), 1: State(labels=['carchosen'], features={}, id=1, model=..., observation=None, name='1'), 2: State(labels=['carchosen'], features={}, id=2, model=..., observation=None, name='2'), 3: State(labels=['carchosen'], features={}, id=3, model=..., observation=None, name='3'), 4: State(labels=['open'], features={}, id=4, model=..., observation=None, name='4'), 5: State(labels=['open'], features={}, id=5, model=..., observation=None, name='5'), 6: State(labels=['open'], features={}, id=6, model=..., observation=None, name='6'), 7: State(labels=['open'], features={}, id=7, model=..., observation=None, name='7'), 8: State(labels=['open'], features={}, id=8, model=..., observation=None, name='8'), 9: State(labels=['open'], features={}, id=9, model=..., observation=None, name='9'), 10: State(labels=['open'], features={}, id=10, model=..., observation=None, name='10'

We can visualize our model together with our results

In [7]:
from stormvogel.layout import Layout
vis = show.show(mdp, result=result, show_editor=True, scheduler=result.scheduler)

Output()

Output()

Output()

To learn how to create more elaborate property strings, visit: https://www.stormchecker.org/documentation/background/properties.html

In [10]:
result.scheduler.taken_actions

{0: Action(labels=frozenset({'0'})),
 1: Action(labels=frozenset({'open1'})),
 2: Action(labels=frozenset({'2'})),
 3: Action(labels=frozenset({'2'})),
 4: Action(labels=frozenset({'0'})),
 5: Action(labels=frozenset({'0'})),
 6: Action(labels=frozenset({'0'})),
 7: Action(labels=frozenset({'0'})),
 8: Action(labels=frozenset({'0'})),
 9: Action(labels=frozenset({'0'})),
 10: Action(labels=frozenset({'0'})),
 11: Action(labels=frozenset({'0'})),
 12: Action(labels=frozenset({'0'})),
 13: Action(labels=frozenset({'0'})),
 14: Action(labels=frozenset({'0'})),
 15: Action(labels=frozenset({'0'})),
 16: Action(labels=frozenset({'0'})),
 17: Action(labels=frozenset({'0'})),
 18: Action(labels=frozenset({'0'})),
 19: Action(labels=frozenset({'0'})),
 20: Action(labels=frozenset({'0'})),
 21: Action(labels=frozenset({'0'})),
 22: Action(labels=frozenset({'0'})),
 23: Action(labels=frozenset({'0'})),
 24: Action(labels=frozenset({'0'})),
 25: Action(labels=frozenset({'0'})),
 26: Action(labels